# Traffic Lights Route Planner

Use this notebook to plan a route and visualize traffic lights along the way. Enter start and end addresses below.

In [ ]:
import json
import requests
from pathlib import Path
import folium
from geopy.geocoders import Nominatim
import gpxpy
from ipywidgets import Text, Button, VBox, Output
from IPython.display import display

TRAFFIC_LIGHTS_FILE = Path('data/traffic_lights.geojson')

geolocator = Nominatim(user_agent='traffic_lights_route')

def geocode(address: str):
    location = geolocator.geocode(address)
    if not location:
        raise ValueError(f'Could not geocode address: {address}')
    return (location.latitude, location.longitude)

def get_route(start, end):
    url = f"https://router.project-osrm.org/route/v1/driving/{start[1]},{start[0]};{end[1]},{end[0]}?overview=full&geometries=geojson"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    coords = data['routes'][0]['geometry']['coordinates']
    return [(lat, lon) for lon, lat in coords]

def load_traffic_lights(path=TRAFFIC_LIGHTS_FILE):
    path = Path(path)
    if not path.exists():
        return []
    lights = []
    if path.suffix.lower() in {'.geojson', '.json'}:
        data = json.loads(path.read_text())
        for feature in data.get('features', []):
            lon, lat = feature['geometry']['coordinates']
            name = feature['properties'].get('name', 'Traffic Light')
            lights.append((lat, lon, name))
    elif path.suffix.lower() == '.gpx':
        with path.open() as f:
            gpx = gpxpy.parse(f)
        for wp in gpx.waypoints:
            lights.append((wp.latitude, wp.longitude, wp.name or 'Traffic Light'))
    else:
        raise ValueError('Unsupported file type')
    return lights

start_box = Text(description='Start:')
end_box = Text(description='End:')
button = Button(description='Show Route')
output = Output()

def on_click(_):
    output.clear_output()
    with output:
        start = geocode(start_box.value)
        end = geocode(end_box.value)
        route = get_route(start, end)
        m = folium.Map(location=start, zoom_start=13)
        folium.PolyLine(route, color='blue', weight=5).add_to(m)
        for lat, lon, name in load_traffic_lights():
            folium.Marker((lat, lon), popup=name, icon=folium.Icon(color='red', icon='traffic-light')).add_to(m)
        display(m)

button.on_click(on_click)

VBox([start_box, end_box, button, output])
